In [2]:
#______________________________________________________________________________________________
#Install Tensorflow and Keras
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)
#_____________________________________________________________________________________________
#
!pip uninstall tf-keras
!pip install tensorflow==2.16.1

from keras import backend
print(backend.backend())



# ──────────────────────────────────────────────────────────────────────────────
# 1) Gerek­li Kutuphane ve Modüllerin İmport Edilmesi
# ──────────────────────────────────────────────────────────────────────────────
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ──────────────────────────────────────────────────────────────────────────────
# 2) Veri Ön­iş­le­me Fonksiyonu
#    • CSV’den oku
#    • MinMaxScaler ile [0,1] aralığına ölçeklendir
#    • Özellik (X) ve etiket (y) matrislerini döndür
# ──────────────────────────────────────────────────────────────────────────────
data_path = '/content/drive/MyDrive/diabetes.csv adlı dosyanın kopyası'
def preprocess_data(filepath):
    data = pd.read_csv(filepath)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(data.drop('Outcome', axis=1))
    y = data['Outcome'].values
    return X, y

# Örnek: diabetes.csv dosyasını kullanarak veriyi hazırla
X, y = preprocess_data('/content/drive/MyDrive/diabetes.csv adlı dosyanın kopyası')

# ──────────────────────────────────────────────────────────────────────────────
# 3) Train–Validation Ayrımı
#    • %80 eğitim, %20 doğrulama
#    • Sınıf dengesini korumak için stratify=y
# ──────────────────────────────────────────────────────────────────────────────
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("X_train:", X_train.shape, "X_val:", X_val.shape)

# ──────────────────────────────────────────────────────────────────────────────
# 4) tf.data Dataset Oluşturma
#    • from_tensor_slices ile dönüştür
#    • shuffle + batch + prefetch ile pipeline hazırla
# ──────────────────────────────────────────────────────────────────────────────
train_ds = (
    tf.data.Dataset
      .from_tensor_slices((X_train, y_train))
      .shuffle(buffer_size=len(X_train))
      .batch(32)
      .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset
      .from_tensor_slices((X_val, y_val))
      .batch(32)
      .prefetch(tf.data.AUTOTUNE)
)

# ──────────────────────────────────────────────────────────────────────────────
# 5) TensorFlow Dataset’ten Örnek İnceleme
#    • Birinci gözlemi yazdır
#    • 10. batch’in features & labels değerlerini incele
# ──────────────────────────────────────────────────────────────────────────────
print("\n-- Val setinden 1. gözlem --")
for feat, lab in val_ds.take(1):
    print("Features:", feat.numpy()[0], "\nLabel:", lab.numpy()[0])

print("\n-- Val seti 10. batch (indeks=9) --")
for idx, (feat_batch, lab_batch) in enumerate(val_ds):
    if idx == 9:
        print("Features batch[0]:", feat_batch.numpy()[0])
        print("Labels batch[0]:", lab_batch.numpy()[0])
        break

# ──────────────────────────────────────────────────────────────────────────────
# 6) Ders Modeli ile 100 Epoch Eğitim
#    • 2 katmanlı basit model
#    • Adam, binary_crossentropy, accuracy
#    • Son val_loss / val_accuracy yazdır ve yorum satırı bırak
# ──────────────────────────────────────────────────────────────────────────────
model_simple = Sequential([
    Input(shape=(8,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8,  activation='relu'),
    Dense(1,  activation='sigmoid'),
])
model_simple.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_simple = model_simple.fit(
    train_ds, validation_data=val_ds,
    epochs=100, verbose=0
)
val_loss_100 = history_simple.history['val_loss'][-1]
val_acc_100  = history_simple.history['val_accuracy'][-1]
print(f"\n[Ders Modeli - 100 epoch] val_loss: {val_loss_100:.3f}, val_acc: {val_acc_100:.3f}")
# Yorum: Eğer val_acc ≈ %70’in altında kaldıysa under-fit, %85’i aşarsa over-fit riski var.

# ──────────────────────────────────────────────────────────────────────────────
# 7) Derin Model: 5 Katman, [32,64,128,256,512] Nöron
#    • Aynı optimizer/loss/metric
#    • Epoch=100, yorum bırak
# ──────────────────────────────────────────────────────────────────────────────
model_deep = Sequential([
    Input(shape=(8,)),
    Dense(32,  activation='relu'),
    Dense(64,  activation='relu'),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1,   activation='sigmoid'),
])
model_deep.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_deep.summary()
history_deep = model_deep.fit(
    train_ds, validation_data=val_ds,
    epochs=100, verbose=0
)
val_loss_deep = history_deep.history['val_loss'][-1]
val_acc_deep  = history_deep.history['val_accuracy'][-1]
print(f"\n[Derin Model 5 Katman] val_loss: {val_loss_deep:.3f}, val_acc: {val_acc_deep:.3f}")
# Yorum: Parametre sayısı arttıkça over-fit riski yükselir; regularization gerekebilir.

# ──────────────────────────────────────────────────────────────────────────────
# 8) Uzun Eğitim: 10.000 Epoch + EarlyStopping
#    • patience=50, restore_best_weights=True
#    • Gerçek epoch sayısını ve en iyi val_loss’i yazdır
# ──────────────────────────────────────────────────────────────────────────────
es = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True,
    verbose=0
)
history_long = model_simple.fit(
    train_ds, validation_data=val_ds,
    epochs=10000,
    callbacks=[es],
    verbose=0
)
print(f"\n[Uzun Eğitim] Gerçek epoch: {len(history_long.history['loss'])}, "
      f"En iyi val_loss: {min(history_long.history['val_loss']):.3f}")
# Yorum: EarlyStopping ~150–300 epoch civarında durur; 10000 doğrudan over-fit ve zaman kaybı.


Keras Current Version: 3.8.0 Tensorflow Current Version: 2.16.1
tensorflow
X_train: (614, 8) X_val: (154, 8)

-- Val setinden 1. gözlem --
Features: [0.41176471 0.79899497 0.52459016 0.         0.         0.40834575
 0.09222886 0.31666667] 
Label: 0

-- Val seti 10. batch (indeks=9) --

[Ders Modeli - 100 epoch] val_loss: 0.495, val_acc: 0.721


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 175,841 (686.88 KB)

 Trainable params: 175,841 (686.88 KB)

 Non-trainable params: 0 (0.00 B)


[Derin Model 5 Katman] val_loss: 1.491, val_acc: 0.695

[Uzun Eğitim] Gerçek epoch: 105, En iyi val_loss: 0.492
